In [1]:
# @title Подключение к диску с данными
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title Импорт библиотек , 📂 конфигурация путей и параметров
import polars as pl
from datetime import datetime
import os


DATA_PATH = "/content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/"
PARTS = [1, 2, 3] # Номера частей данных для обработки

# Обработка данных
1. Инициализация LazyFrame
2. Парсим дату и время
3. Загружаем метки (labels)
4. Left join с labels ,
* target = -1 для операций без обратной связи (где нет label)
5. collect() выполняет граф вычислений и возвращает DataFrame
6. Вывод статистики

In [3]:
print("🚀 Начинаем процесс обработки данных...")

for i in PARTS:
    print(f"\n{'='*40}")
    print(f"📦 Обработка части {i} из {len(PARTS)}")
    print(f"{'='*40}")

    input_path = f"{DATA_PATH}train_part_{i}.parquet"
    output_path = f"{DATA_PATH}train_full_part_{i}.parquet"
    labels_path = f"{DATA_PATH}train_labels.parquet"

    # 1️⃣ Инициализация LazyFrame
    # Используем scan_parquet для ленивой загрузки
    q = pl.scan_parquet(input_path)

    # 2️⃣ Преобразование данных
    # Парсим дату и время
    q = q.with_columns(
        pl.col("event_dttm").str.to_datetime("%Y-%m-%d %H:%M:%S")
    )

    # Загружаем метки (labels)
    # Здесь используем read_parquet, так как файл небольшой
    labels = pl.scan_parquet(labels_path)

    # 3️⃣ Объединение (Join) и обработка пропусков
    # Left join с labels
    # target = -1 для операций без обратной связи (где нет label)
    q = q.join(
        labels.select(["event_id", "target"]), # Берем только нужные колонки
        on="event_id",
        how="left"
    ).with_columns(
        pl.col("target").fill_null(-1).cast(pl.Int32)
    )

    # 4️⃣ Выполнение и сохранение
    # collect() выполняет граф вычислений и возвращает DataFrame
    print("⚙️ Выполняем вычисления и загружаем в память...")
    df = q.collect()

    print(f"💾 Сохраняем в {output_path}...")
    df.write_parquet(output_path, compression="zstd")

    # 5️⃣ Вывод статистики
    print("\n📊 Статистика:")
    print(f"   🔹 Размерность (строки, колонки): {df.shape}")

    # Красивый вывод распределения классов
    target_counts = df["target"].value_counts().sort("target")
    print("   🔹 Распределение классов (target):")
    for row in target_counts.iter_rows():
        target_val = row[0]
        count = row[1]
        print(f"      🏷️  Класс {target_val}: {count:,} шт.")

    del df

print(f"\n✨ Готово! Все части обработаны.")

🚀 Начинаем процесс обработки данных...

📦 Обработка части 1 из 3
⚙️ Выполняем вычисления и загружаем в память...
💾 Сохраняем в /content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/train_full_part_1.parquet...

📊 Статистика:
   🔹 Размерность (строки, колонки): (28618594, 24)
   🔹 Распределение классов (target):
      🏷️  Класс -1: 28,589,128 шт.
      🏷️  Класс 0: 12,082 шт.
      🏷️  Класс 1: 17,384 шт.

📦 Обработка части 2 из 3
⚙️ Выполняем вычисления и загружаем в память...
💾 Сохраняем в /content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/train_full_part_2.parquet...

📊 Статистика:
   🔹 Размерность (строки, колонки): (28558397, 24)
   🔹 Распределение классов (target):
      🏷️  Класс -1: 28,529,219 шт.
      🏷️  Класс 0: 12,021 шт.
      🏷️  Класс 1: 17,157 шт.

📦 Обработка части 3 из 3
⚙️ Выполняем вычисления и загружаем в память...
💾 Сохраняем в /content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/train_full_part_3.parquet...

📊 Статистика:
   🔹 Размерность (строки, колонки): (